# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d ukfood -c establishments --drop --jsonArray establishments.json`

In [195]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from decimal import Decimal

In [196]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [197]:
# confirm that our new database was created
databases = mongo.list_database_names()
print('Databases:', databases)

Databases: ['admin', 'classDB', 'config', 'local', 'travel_db', 'ukfood']


In [198]:
# assign the uk_food database to a variable name
db = mongo['ukfood']

In [199]:
# review the collections in our new database
collections = db.list_collection_names()
print('Collections:', collections)

Collections: ['establishments']


In [200]:
# assign the collection to a variable
establishments = db['establishments']

In [201]:
# review a document in the establishments collection
documentone = establishments.find_one()
pprint(documentone)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6673e65b88844bef0205d019'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [202]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [203]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [204]:
# Check that the new restaurant was inserted
print(f"New restaurant inserted with id: {result.inserted_id}")

New restaurant inserted with id: 6673e685e2ca0aa394076262


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [205]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
querybtid = {"BusinessType": "Restaurant/Cafe/Canteen"}
fieldselect = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
businesstypeid = establishments.find_one(querybtid, fieldselect)
bustypeid = businesstypeid["BusinessTypeID"]
pprint(bustypeid)

1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [206]:
# Update the new restaurant with the correct BusinessTypeID
queryPF = {"BusinessName": "Penang Flavours"}
update = {"$set": {"BusinessTypeID": bustypeid}}

result = establishments.update_one(queryPF, update)

pprint(result)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [207]:
# Confirm that the new restaurant was updated
pfverify_query = {"BusinessName": "Penang Flavours"}
pfcheck = establishments.find_one(pfverify_query)
pprint(pfcheck)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6673e685e2ca0aa394076262'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [208]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(dover_query)
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [209]:
# Delete all documents where LocalAuthorityName is "Dover"
removed = establishments.delete_many(dover_query)
print(f"Number of documents deleted: {removed.deleted_count}")

Number of documents deleted: 994


In [210]:
# Check if any remaining documents include Dover
remaining_document = establishments.find_one(dover_query)
pprint(remaining_document)

None


In [211]:
# Check that other documents remain with 'find_one'
pprint(documentone)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6673e65b88844bef0205d019'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [212]:
# Change the data type from String to Decimal for longitude and latitude
latlongquery = {"geocode.longitude": {"$type": "string"}, "geocode.latitude": {"$type": "string"}}
latlongdocs = establishments.find(latlongquery)

for document in latlongdocs:
    try:
        longitude = float(document["geocode"]["longitude"])
        latitude = float(document["geocode"]["latitude"])
        
        # Update the document with the converted float values
        establishments.update_one(
            {"_id": document["_id"]},
            {"$set": {"geocode.longitude": longitude, "geocode.latitude": latitude}}
        )
    except ValueError:
        # Handle the case where the Longitude or Latitude cannot be converted to a float
        print(f"Could not convert Geocode.Longitude or Geocode.Latitude for document with _id: {document['_id']}")

# Print a confirmation message
print("Longitude and Latitude data types have been updated to double.")

Longitude and Latitude data types have been updated to double.


Use `update_many` to convert `RatingValue` to integer numbers.

In [213]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [214]:
# Change the data type from String to Integer for RatingValue
ratingquery = {"RatingValue": {"$type": "string"}}
ratingdocs = establishments.find(ratingquery)

for document in ratingdocs:
    try:
        rating_value = int(document["RatingValue"])
        
        establishments.update_one(
            {"_id": document["_id"]},
            {"$set": {"RatingValue": rating_value}}
        )
    except ValueError:
        print(f"Could not convert RatingValue for document with _id: {document['_id']}")

print("RatingValue data types have been updated to Integer.")

RatingValue data types have been updated to Integer.


In [215]:
# Check that the coordinates and rating value are now numbers

coordinates_query = {
    "geocode.longitude": {"$type": "double"},  
    "geocode.latitude": {"$type": "double"}   
}

coordinates_count = establishments.count_documents(coordinates_query)

rating_value_query = {
    "RatingValue": {"$type": "int"} 
}

rating_value_count = establishments.count_documents(rating_value_query)

print(f"Number of documents with Longitude and Latitude as double: {coordinates_count}")
print(f"Number of documents with RatingValue as integer: {rating_value_count}")


Number of documents with Longitude and Latitude as double: 38786
Number of documents with RatingValue as integer: 34694
